In [16]:
pd.set_option('display.max_columns', None)

In [1]:
import pandas as pd
import numpy as np

path = "DB-Output_original.csv"
data = pd.read_csv(path)

data = data.loc[:, :'Initial Solution']
data

,Instance's Name,Initial Solution
0,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
1,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
2,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
3,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
4,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
...,...,...
35406,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."
35407,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."
35408,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."
35409,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."


In [2]:
init_sol = data['Initial Solution']
print("Type of 'init_sol':", type(init_sol))

init_sol[0]
print("Type of 'init_sol[0]':", type(init_sol[0]))

init_sol.head()

Type of 'init_sol': <class 'pandas.core.series.Series'>
Type of 'init_sol[0]': <class 'str'>


0    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
1    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
2    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
3    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
4    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
Name: Initial Solution, dtype: object

Somma delle occorrenza di uno specifico luogo all'interno di tutto il DataFrame, nella colonna "Initial Solution"

In [3]:
print("D0 total: ", init_sol.str.count('D0').sum())
print("C21 total: ", init_sol.str.count('C21').sum())
print("C87 total: ", init_sol.str.count('C87').sum())
print("C27 total: ", init_sol.str.count('C27').sum())

D0 total:  1817188
C21 total:  24067
C87 total:  18864
C27 total:  24666


# Pulizia dei dati

Si prende la colonna contenente i luoghi "Initial Solution" (che è una serie di pandas) e tramite la funzione str.replace() si pulisce la stringa, rimuovendo la punteggiatura speciale (parentesi quadre, virgole e apici) con un carattere null.

Si stampa poi la colonna "Initial Solution" ripulita.

In [4]:
data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
data["Initial Solution"] = data["Initial Solution"].str.replace("]","")
data["Initial Solution"] = data["Initial Solution"].str.replace(",","")
data["Initial Solution"] = data["Initial Solution"].str.replace("'","")

data["Initial Solution"].head()


/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_8668/3585595387.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_8668/3585595387.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("]","")


0    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
1    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
2    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
3    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
4    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
Name: Initial Solution, dtype: object

# Divisione dei luoghi

Si "isola" la singola colonna "Initial Solution" inserendola nella variabile "init_sol", che diventa una serie a sua volta.

In [5]:
init_sol = data["Initial Solution"]
print(init_sol)
type(init_sol)

0        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
1        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
2        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
3        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
4        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
                               ...                        
35406    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35407    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35408    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35409    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35410    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
Name: Initial Solution, Length: 35411, dtype: object


pandas.core.series.Series

La singola riga della serie equivale ad una stringa. Ad esempio "init_sol[0]" equivale ad una stringa contenente "D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 ..."

Attraverso il metodo split(), inserito in un ciclo for che itera ogni riga di "init_sol", si splitta la stringa in considerazione rispetto al carattere spazio e la si inserisce tramite .append() nella lista "places" inizialmente vuota. Ogni volta che il metodo split() trova uno spazio, viene rilevato un elemento (luogo).

La stringa splittata per singolo elemento viene inserita in una lista chiamata "places", dove sono presenti ancora ripetizioni.

Applicando il cast di tipo da lista a set, e di nuovo da set a lista, si ha in output una lista senza ripetizioni chiamata "distinct_places".
Successivamente, attraverso il metodo .sort(), la lista viene ordinata in modo alfabetico.

In [6]:
places = []
for row in init_sol:
    for elem in row.split():
        places.append(elem)

distinct_places = list(set(places))

distinct_places.sort()
print("\nLa lista di tutti i diversi luoghi (ordinati) è:\n", distinct_places)

print("\nLa lista contiene", len(distinct_places), "diversi luoghi")
print("\nPrint di diversi luoghi:\n", distinct_places[0], distinct_places[5], distinct_places[23])



La lista di tutti i diversi luoghi (ordinati) è:
 ['C1', 'C10', 'C100', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C3', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C4', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C5', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C6', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C7', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C8', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C9', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'D0', 'S0', 'S1', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S2', 'S20', 'S3', 'S4', 'S5', 'S7', 'S9']

La lista contiene 120 diversi luoghi

Print di diversi luoghi:
 C1 C13 C3


In [83]:
distinct_places.index("C11")

3

# Creazione della matrice 

Si procede quindi alla creazione della matrice che ha come dimensioni la lunghezza della lista "distinct_places", ovvero il numero dei diversi luoghi presenti nel dataframe iniziale 

In [7]:
print("\nLa lista 'distinct_places' contiene", len(distinct_places), "diversi luoghi\n")

pd.set_option('display.max_columns', None)
matrix = pd.DataFrame(0, index=distinct_places, columns=distinct_places)
matrix.head(105)

# usiamo un dataframe per avere indici secondo i quali andare ad inserire le corrispondenze, 
# successivamente si converte in matrice numpy


La lista 'distinct_places' contiene 120 diversi luoghi



,C1,C10,C100,C11,C12,C13,C14,C15,C16,C17,C18,C19,C2,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C3,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C4,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C5,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C6,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C7,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C8,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C9,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,D0,S0,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S3,S4,S5,S7,S9
C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
S1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
S10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# metodo 1
index = 0            # Python's indexing starts at zero
for item in places:   # Python's for loops are a "for each" loop 
    print(index, item)
    index += 1
    
    
# metodo 2  
for i, j in enumerate(places):
    print(i, j)
    

for index, item in enumerate(items, start=0):   # default is zero
    print(index, item)

In [10]:
a = len(distinct_places)
places_matrix = np.zeros((a, a))
print("Matrice dei vari luoghi di dimensione", places_matrix.shape, ":\n", places_matrix)

Matrice dei vari luoghi di dimensione (120, 120) :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
place_ridotto = places[:119]

In [ ]:
for index, item in enumerate(place_ridotto, start=0):   # default is zero
    print(index, item)

In [15]:
for index, item in enumerate(place_ridotto):
    index_a = index
    index_b = index+1
    
    item_a = place_ridotto[index]
    item_b = place_ridotto[index+1]
    
    print(index_a, index_b, item_a, item_b)
    
    places_matrix[index_a][index_b] = 1
    

0 1 D0 C21
1 2 C21 C26
2 3 C26 D0
3 4 D0 D0
4 5 D0 C65
5 6 C65 C61
6 7 C61 D0
7 8 D0 D0
8 9 D0 C24
9 10 C24 C50
10 11 C50 D0
11 12 D0 D0
12 13 D0 C25
13 14 C25 D0
14 15 D0 D0
15 16 D0 C6
16 17 C6 C91
17 18 C91 D0
18 19 D0 D0
19 20 D0 C9
20 21 C9 D0
21 22 D0 D0
22 23 D0 C2
23 24 C2 D0
24 25 D0 D0
25 26 D0 C90
26 27 C90 C85
27 28 C85 D0
28 29 D0 D0
29 30 D0 C32
30 31 C32 S11
31 32 S11 D0
32 33 D0 D0
33 34 D0 C82
34 35 C82 C81
35 36 C81 S20
36 37 S20 D0
37 38 D0 D0
38 39 D0 C18
39 40 C18 D0
40 41 D0 D0
41 42 D0 C15
42 43 C15 S7
43 44 S7 D0
44 45 D0 D0
45 46 D0 C100
46 47 C100 S3
47 48 S3 D0
48 49 D0 D0
49 50 D0 C16
50 51 C16 S7
51 52 S7 D0
52 53 D0 D0
53 54 D0 C97
54 55 C97 S3
55 56 S3 D0
56 57 D0 D0
57 58 D0 C39
58 59 C39 S12
59 60 S12 D0
60 61 D0 D0
61 62 D0 C94
62 63 C94 S4
63 64 S4 D0
64 65 D0 D0
65 66 D0 C58
66 67 C58 S16
67 68 S16 D0
68 69 D0 D0
69 70 D0 C73
70 71 C73 S20
71 72 S20 D0
72 73 D0 D0
73 74 D0 C21
74 75 C21 C26
75 76 C26 D0
76 77 D0 D0
77 78 D0 C65
78 79 C65 C61
79 80 C6

IndexError: list index out of range

In [69]:
places_matrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [88]:
luoghi = places[:4]
indice = list(set(luoghi))
indice.sort()

# INIZIO

In [102]:
print("lista dei singoli luoghi senza ripetizioni: LUOGHI =>", luoghi)
print("\nlista indice matrice: INDICE =>", indice)

lista dei singoli luoghi senza ripetizioni: LUOGHI => ['D0', 'C21', 'C26', 'D0']

lista indice matrice: INDICE => ['C21', 'C26', 'D0']


In [103]:
array = np.zeros((3, 3))
array

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [117]:
for index, elem in enumerate(luoghi):
    print(index == indice.index("C21"))

True
False
False
False
